In [1]:
import os
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
import numpy as np
train = pd.read_csv('kc_house_data.csv') 
print('Shape of the train data with all features:', train.shape)
train = train.select_dtypes(exclude=['object'])
print('Shape of the train data with numerical features:', train.shape)
train.drop('id',axis = 1, inplace = True)
train.fillna(0,inplace=True)

Shape of the train data with all features: (21613, 21)
Shape of the train data with numerical features: (21613, 20)


In [2]:
from sklearn.ensemble import IsolationForest

clf = IsolationForest(max_samples = 100, random_state = 42)
clf.fit(train)
y_noano = clf.predict(train)
y_noano = pd.DataFrame(y_noano, columns = ['Top'])
y_noano[y_noano['Top'] == 1].index.values

train = train.iloc[y_noano[y_noano['Top'] == 1].index.values]
train.reset_index(drop = True, inplace = True)
print("Number of Outliers:", y_noano[y_noano['Top'] == -1].shape[0])
print("Number of rows without outliers:", train.shape[0])

Number of Outliers: 2805
Number of rows without outliers: 18808


In [3]:
col_train = list(train.columns)
X_train = train.drop('price', axis = 1)
y_train = np.array(train.price).reshape((18808,1))
prepro = MinMaxScaler()
prepro.fit(X_train)
X_train = np.array(prepro.transform(X_train))

In [4]:
def build_model():
    model = keras.models.Sequential()

    model.add(keras.layers.Dense(200, input_shape=(18,), kernel_initializer='normal', activation='relu'))
    model.add(keras.layers.Dense(100, kernel_initializer='normal', activation='relu'))
    model.add(keras.layers.Dense(50, kernel_initializer='normal', activation='relu'))
    model.add(keras.layers.Dense(25, kernel_initializer='normal', activation='relu'))
    model.add(keras.layers.Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adadelta(), metrics=["mse"])
     
    return model 

In [ ]:
k = 10
num_val = len(X_train) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    X_val = X_train[i * num_val: (i + 1) * num_val]
    Y_val = y_train[i * num_val: (i + 1) * num_val]

    # Prepare the training data: data from all other partitions
    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    Y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    # Build the Keras model (already compiled)
    model = build_model()
    # Train the model (in silent mode, verbose=0)
    model.fit(X_train_part, Y_train_part,epochs=num_epochs, verbose=0)
    # Evaluate the model on the validation data 
    val_loss, val_acc = model.evaluate(X_val, Y_val, verbose=0)
    all_scores.append(val_acc)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
processing fold # 5


In [ ]:
np.mean(all_scores)